In [ ]:
from navground import sim
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np

import iblofunmatch.inter as ibfm

from navground import sim, core

In [ ]:
side_length = 20
yaml = f"""
steps: 10000
time_step: 0.1
record_safety_violation: true
record_task_events: true
record_pose: true
runs: 2
scenario:
  type: Cross
  side: {side_length}
  agent_margin: 0.2
  add_safety_to_agent_margin: true
  tolerance: 0.6
  position_noise: 0.2
  groups:
    - number: 25
      type: thymio
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.1
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0
"""
scenario = sim.load_scenario(yaml)

In [ ]:
import random

world = sim.World()
scenario.init_world(world, seed=random.randint(0, 2**31))

In [ ]:
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
ps = experiment.runs[0].poses

### Compute Block Function between two timesteps

In [ ]:
start_time = 50
end_time = 100 # long wait
X = ps[start_time][:,[0,1]]
Y = ps[end_time][:,[0,1]]
unique_pts = np.sum(np.abs(X-Y), axis=1)>10e-7
repeated_pts = np.nonzero(unique_pts==False)[0].tolist()
Y_unique = Y[unique_pts]
Z = np.vstack((X, Y_unique))
output_dir="output"
os.makedirs(output_dir, exist_ok=True)
ibfm_bfun = [] 
for i in range(2):
    if i ==0:
        indices_subpoints = list(range(X.shape[0]))
    elif i==1:
        indices_subpoints = list(range(X.shape[0], Z.shape[0]))
        indices_subpoints += repeated_pts
    # finished with lists 
    print(indices_subpoints)
    ibfm_bfun.append(ibfm.get_IBloFunMatch_subset(None, Z, indices_subpoints, output_dir, max_rad=-1, num_it=4, points=True))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ibfm.plot_XYZ_matching_0(ibfm_bfun, ax)
ax.set_xlim([-4,4])
plt.savefig("matching_0_dim_large_radius.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ibfm.plot_XYZ_matching_0(ibfm_bfun, ax)
ax.set_xlim([-4,4])
plt.savefig("matching_0_dim_large_margin.png")

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(18,6))
colorlist = ["red", "black", "blue"]
# Scatter points 
ax[0].scatter(X[:,0], X[:,1], c=colorlist[0])
ax[2].scatter(Y[:,0], Y[:,1], c=colorlist[2])
# plot both point clouds in the middle
ax[1].scatter(X[:,0], X[:,1], c=colorlist[0], s=30, zorder=3, marker="x")
ax[1].scatter(Y[:,0], Y[:,1], c=colorlist[2], s=30, zorder=3, marker="+")
for start, end in zip(X, Y):
    ax[1].plot([start[0], end[0]], [start[1], end[1]], c="black")
plt.savefig("points_matching_low_distance.png")

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(18,6))
colorlist = ["red", "black", "blue"]
# Scatter points 
ax[0].scatter(X[:,0], X[:,1], c=colorlist[0])
ax[2].scatter(Y[:,0], Y[:,1], c=colorlist[2])
# plot both point clouds in the middle
ax[1].scatter(X[:,0], X[:,1], c=colorlist[0], s=30, zorder=3, marker="x")
ax[1].scatter(Y[:,0], Y[:,1], c=colorlist[2], s=30, zorder=3, marker="+")
for start, end in zip(X, Y):
    ax[1].plot([start[0], end[0]], [start[1], end[1]], c="black")
plt.savefig("points_matching_large_margin.png")

In [ ]:
block_00 = ibfm_bfun[0]["block_function_0"]
block_10 = ibfm_bfun[1]["block_function_0"]
unmatched_block_0 = [i for i in range(ibfm_bfun[0]["X_barcode_0"].shape[0]) if (i not in block_00) and (i not in block_10)]
double_matched_block_0 = [i for i in block_00 if i in block_10]
# unmatched dim 1 
block_01 = ibfm_bfun[0]["block_function_1"]
block_11 = ibfm_bfun[1]["block_function_1"]
unmatched_block_1 = [i for i in range(ibfm_bfun[0]["X_barcode_1"].shape[0]) if (i not in block_01) and (i not in block_11)]

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=4, figsize=(20,5))
ibfm.plot_matching(ibfm_bfun[0], ax[[0,1]], fig, max_rad=-1, block_function=True, dim=0, codomain_int=unmatched_block_0, repeated_codomain=double_matched_block_0)
ibfm.plot_matching(ibfm_bfun[1], ax[[2,3]], fig, max_rad=-1, block_function=True, dim=0, codomain_int=unmatched_block_0, repeated_codomain=double_matched_block_0)
plt.savefig("block_functions_large_margin.png")

In [ ]:
common_cycles = np.intersect1d(ibfm_bfun[0]["block_function_1"], ibfm_bfun[1]["block_function_1"])
common_cycles = common_cycles[common_cycles>=0].tolist()
common_cycles

In [ ]:
common_cycles = np.intersect1d(ibfm_bfun[0]["block_function_1"], ibfm_bfun[1]["block_function_1"])
common_cycles = common_cycles[common_cycles>=0].tolist()
fig, ax = plt.subplots(nrows=len(common_cycles), ncols=3, figsize=(18,6*len(common_cycles)))
if len(common_cycles)==1:
    ax = [ax]
for j, idx in enumerate(common_cycles):
    idx_X = ibfm_bfun[0]["block_function_1"].index(idx)
    idx_Y = ibfm_bfun[1]["block_function_1"].index(idx)
    assert(ibfm_bfun[0]["block_function_1"][idx_X]==idx)
    assert(ibfm_bfun[1]["block_function_1"][idx_Y]==idx)
    # Scatter points 
    colorlist = ["red", "black", "blue"]
    ax[j][0].scatter(X[:,0], X[:,1], c=colorlist[0])
    ax[j][2].scatter(Y[:,0], Y[:,1], c=colorlist[2])
    # plot both point clouds in the middle
    ax[j][1].scatter(Z[:,0], Z[:,1], c=colorlist[1], s=30, zorder=2)
    ax[j][1].scatter(X[:,0], X[:,1], c=colorlist[0], s=30, zorder=3, marker="x")
    ax[j][1].scatter(Y[:,0], Y[:,1], c=colorlist[2], s=30, zorder=3, marker="+")
    cycle_list = [ibfm_bfun[0]["S_reps_1"][idx_X], ibfm_bfun[1]["X_reps_1"][idx], ibfm_bfun[1]["S_reps_1"][idx_Y]]
    for i, cycle in enumerate(cycle_list):
        repr = cycle.copy()
        while len(repr)>0:
            edge = [repr.pop(), repr.pop()]
            ax[j][i].plot(Z[edge][:,0], Z[edge][:,1], linewidth=3, c=colorlist[i])

plt.savefig("cycles_match.png")